In [1]:
from bs4 import BeautifulSoup
import requests
import smtplib #library that helps sending emails to yourself
import time
import datetime



In [2]:
# Connect to Website and pull in data

URL = 'https://www.amazon.com/dp/B08T92MZ2Y/ref=sspa_dk_detail_3?pd_rd_i=B08T92MZ2Y&pd_rd_w=IiJ7K&content-id=amzn1.sym.eb7c1ac5-7c51-4df5-ba34-ca810f1f119a&pf_rd_p=eb7c1ac5-7c51-4df5-ba34-ca810f1f119a&pf_rd_r=HZC1DA12M2H2J7HN18FN&pd_rd_wg=aSHtY&pd_rd_r=aadce9f2-56b7-49f8-954b-aecb673afa9f&s=apparel&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWw&th=1&psc=1'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

In [3]:

title = soup2.find(id='productTitle').get_text()
title=title.strip()
print(title)

price=soup2.find(class_='a-section aok-hidden twister-plus-buying-options-price-data').get_text() #this store price as a string
price=price[61:66]
print(price)

ratings=soup2.find('span',class_='a-icon-alt').get_text()
x=ratings.strip()
ratings=x[0:3]
print(ratings)
    

People Who Tolerate Me on a Daily Basis T-Shirt, Funny Tshirt for Men, Sarcastic Graphic Novelty T Shirt
19.99
4.7


In [4]:
#repeating this to do this so that we can extract these values for this item over time and export this data in a csv 
#getting date of when the data is collected
today=datetime.date.today()

import csv
header=['Date','Title','Price','Ratings']
data=[today,title,price,ratings]
type(data)

with open('Amazon_product_data.csv','w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)
    

In [5]:
import pandas as pd
df=pd.read_csv(r"C:\Users\swati\Amazon_product_data.csv")
df

,Date,Title,Price,Ratings
0,2023-11-29,People Who Tolerate Me on a Daily Basis T-Shir...,19.99,4.7


In [6]:
#aim is to check how price of this one itme varies with time
with open('Amazon_product_data.csv','a+',newline='',encoding='UTF8')as f: #a+ appends data
    writer=csv.writer(f)
    writer.writerow(data)

In [11]:
#automating the above code so that it mails the data everyday automatically without manually running  the code

#putting all of the above code in a function

def check_price():
    # Connect to Website and pull in data

    URL = 'https://www.amazon.com/dp/B08T92MZ2Y/ref=sspa_dk_detail_3?pd_rd_i=B08T92MZ2Y&pd_rd_w=IiJ7K&content-id=amzn1.sym.eb7c1ac5-7c51-4df5-ba34-ca810f1f119a&pf_rd_p=eb7c1ac5-7c51-4df5-ba34-ca810f1f119a&pf_rd_r=HZC1DA12M2H2J7HN18FN&pd_rd_wg=aSHtY&pd_rd_r=aadce9f2-56b7-49f8-954b-aecb673afa9f&s=apparel&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWw&th=1&psc=1'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    title = soup2.find(id='productTitle').get_text()
    title=title.strip()
    
    price=soup2.find(class_='a-section aok-hidden twister-plus-buying-options-price-data').get_text() #this store price as a string
    price=price[61:66]
    
    ratings=soup2.find('span',class_='a-icon-alt').get_text()
    ratings=ratings.strip()
    ratings=ratings[0:3]
    
    today=datetime.date.today()
    
    import csv
    header=['Date','Title','Price','Ratings']
    data=[today,title,price,ratings]
    
    with open('Amazon_product_data.csv','a+',newline='',encoding='UTF8')as f: #a+ appends data
        writer=csv.writer(f)
        writer.writerow(data)
    if price <100:
        send_mail()


In [12]:
# Runs check_price after a set time and inputs data into your CSV

while(True):
    check_price()
    time.sleep(86000) #runs everyday

TypeError: '<' not supported between instances of 'str' and 'int'

In [13]:
import pandas as pd
df=pd.read_csv(r"C:\Users\swati\Amazon_product_data.csv")
df

,Date,Title,Price,Ratings
0,2023-11-29,People Who Tolerate Me on a Daily Basis T-Shir...,19.99,4.7
1,2023-11-29,People Who Tolerate Me on a Daily Basis T-Shir...,19.99,4.7
2,2023-11-29,People Who Tolerate Me on a Daily Basis T-Shir...,19.99,4.7
3,2023-11-29,People Who Tolerate Me on a Daily Basis T-Shir...,19.99,4.7


In [14]:
# If uou want to try sending yourself an email (just for fun) when a price hits below a certain level you can try it
# out with this script

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('swatisiyer1209@gmail.com','Sudha_1108#')
    
    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = "Swati, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/dp/B08T92MZ2Y/ref=sspa_dk_detail_3?pd_rd_i=B08T92MZ2Y&pd_rd_w=IiJ7K&content-id=amzn1.sym.eb7c1ac5-7c51-4df5-ba34-ca810f1f119a&pf_rd_p=eb7c1ac5-7c51-4df5-ba34-ca810f1f119a&pf_rd_r=HZC1DA12M2H2J7HN18FN&pd_rd_wg=aSHtY&pd_rd_r=aadce9f2-56b7-49f8-954b-aecb673afa9f&s=apparel&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWw&th=1&psc=1"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'swatisiyer1209@gmail.com',
        msg
     
    )